In [5]:
# This notebook illustrates improvements to scalability gained by point-based evaluation

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman import FunmanWorkRequest, EncodingSchedule 
import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
import pandas as pd

RESOURCES = "../resources"
SAVED_RESULTS_DIR = "./out"

# EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet","amr-examples")
EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet","evaluation")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR, "sir.json"
)
# REQUEST_PATH = os.path.join(
#     EXAMPLE_DIR, "sir_request_param_synth.json"
# )
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "sir_request_consistency.json"
)


request_params = {}

# %load_ext autoreload
# %autoreload 2

In [2]:
# Helper functions to setup FUNMAN for different steps of the scenario

def get_request():
    with open(REQUEST_PATH, "r") as request:
        funman_request = FunmanWorkRequest.model_validate(json.load(request))
        return funman_request

def set_timepoints(funman_request, num_steps, step_size):
    funman_request.structure_parameters[0].interval.lb = num_steps
    funman_request.structure_parameters[0].interval.ub = num_steps
    funman_request.structure_parameters[1].interval.lb = step_size
    funman_request.structure_parameters[1].interval.ub = step_size
    
    # funman_request.structure_parameters[0].schedules = [EncodingSchedule(timepoints=timepoints)]

def unset_all_labels(funman_request):
    for p in funman_request.parameters:
        p.label = "any"
    
def set_all_labels(funman_request):
    for p in funman_request.parameters:
        p.label = "all"    
    
def set_config_options(funman_request, point_based=False, debug=False, dreal_precision=1, prioritize_box_entropy=False):
    # Overrides for configuration
    #
    # funman_request.config.substitute_subformulas = True
    # funman_request.config.use_transition_symbols = True
    # funman_request.config.use_compartmental_constraints=False
    if debug:
        funman_request.config.save_smtlib="./out"
        funman_request.config.verbosity = 10
    funman_request.config.tolerance = 0.1
    funman_request.config.dreal_precision = dreal_precision
    funman_request.config.point_based_evaluation=point_based
    funman_request.config.normalize=False
    funman_request.config.prioritize_box_entropy=prioritize_box_entropy
    
    # funman_request.config.dreal_log_level = "debug"
    # funman_request.config.dreal_prefer_parameters = ["beta","NPI_mult","r_Sv","r_EI","r_IH_u","r_IH_v","r_HR","r_HD","r_IR_u","r_IR_v"]

def get_synthesized_vars(funman_request):
    return [p.name for p in funman_request.parameters if p.label == "all"]

def run(funman_request, plot=False):
    to_synthesize = get_synthesized_vars(funman_request)
    return Runner().run(
        MODEL_PATH,
        funman_request,
        description="SIERHD Eval 12mo Scenario 1 q1",
        case_out_dir=SAVED_RESULTS_DIR,
        dump_plot=plot,
        print_last_time=True,
        parameters_to_plot=to_synthesize
    )

def setup_common(funman_request, num_steps, step_size, point_based=False, synthesize=False, debug=False, dreal_precision=1e-1, prioritize_box_entropy=False):
    set_timepoints(funman_request, num_steps, step_size)
    if not synthesize:
        unset_all_labels(funman_request)
    else:
        set_all_labels(funman_request)
    set_config_options(funman_request, point_based=point_based, debug=debug, dreal_precision=dreal_precision, prioritize_box_entropy=prioritize_box_entropy)
    

def set_compartment_bounds(funman_request, upper_bound=9830000.0, error=0.01):
    # Add bounds to compartments
    for var in STATES:
        funman_request.constraints.append(StateVariableConstraint(name=f"{var}_bounds", variable=var, interval=Interval(lb=0, ub=upper_bound, closed_upper_bound=True),soft=False))

    # Add sum of compartments
    funman_request.constraints.append(LinearConstraint(name=f"compartment_bounds", variables=STATES, additive_bounds=Interval(lb=upper_bound-error, ub=upper_bound+error, closed_upper_bound=False), soft=True))

def relax_parameter_bounds(funman_request, factor = 0.1):
    # Relax parameter bounds
    parameters = funman_request.parameters
    for p in parameters:
        interval = p.interval
        width = float(interval.width())
        interval.lb = interval.lb - (factor/2 * width)
        interval.ub = interval.ub + (factor/2 * width)

def plot_last_point(results):
    pts = results.parameter_space.points() 
    print(f"{len(pts)} points")

    if len(pts) > 0:
        # Get a plot for last point
        df = results.dataframe(points=pts[-1:])
        # pd.options.plotting.backend = "plotly"
        ax = df[STATES].plot()
        
    
        fig = plt.figure()
        # fig.set_yscale("log")
        fig.savefig("save_file_name.pdf")
        plt.close()

def get_last_point_parameters(results):
    pts = results.parameter_space.points()
    if len(pts) > 0:
        pt = pts[-1]
        parameters = results.model._parameter_names()
        param_values = {k:v for k, v in pt.values.items() if k in parameters }
        return param_values

def pretty_print_request_params(params):
    # print(json.dump(params, indent=4))
    if len(params)>0:

        df = pd.DataFrame(params)
        print(df.T)


def report(results, name):
    # plot_last_point(results)
    param_values = get_last_point_parameters(results)
    param_values["runtime"] = results.timing.total_time
    # print(f"Point parameters: {param_values}")
    if param_values is not None:
        request_params[name] = param_values
    pretty_print_request_params(request_params)

def add_unit_test(funman_request):
    pass
    # funman_request.constraints.append(LinearConstraint(name="unit_test", variables = [
    #         "Infected",
    #         "Diagnosed",
    #         "Ailing",
    #         "Recognized",
    #         "Threatened"
    #      ],
    #      additive_bounds= {
    #         "lb": 0.55,
    #         "ub": 0.65
    #      },
    #      timepoints={
    #         "lb": 45,
    #         "ub": 55
    #      }
    #   ))


In [25]:
# Constants for the scenario
STATES = ["S", "I", "R"]

MAX_TIME=250
STEP_SIZE=1
NUM_STEPS=MAX_TIME/STEP_SIZE
timepoints = list(range(0, MAX_TIME+STEP_SIZE, STEP_SIZE))

In [26]:
# Solve using point-based method

funman_request = get_request()
setup_common(funman_request, NUM_STEPS, STEP_SIZE, debug=False, point_based=True, synthesize=True, prioritize_box_entropy=False)
# add_unit_test(funman_request)
results = run(funman_request)
report(results, "point-based")

[0.20154, 0.20154) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.07109, 0.07109) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.99000, 0.99000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.01000, 0.01000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[12.00000, 12.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[12.00000, 12.00000) has equal lower and upper bounds, so assuming the upper boun

Simulation Time: 0:00:02.754625


2024-08-26 02:17:46,450 - funman.server.worker - INFO - Worker.stop() acquiring state lock ....
2024-08-26 02:17:46,872 - funman.server.worker - INFO - FunmanWorker exiting...
2024-08-26 02:17:46,876 - funman.server.worker - INFO - Worker.stop() completed.


Total # of ibex-fwdbwd Pruning                @ Pruning level        =          316260
Total # of ibex-fwdbwd Pruning (zero-effect)  @ Pruning level        =          296057
Total time spent in Pruning                   @ Pruning level        =        0.227849 sec
Total time spent in making constraints        @ Pruning level        =        0.000000 sec
Total # of Convert                            @ Ibex Converter       =            1255
Total time spent in Converting                @ Ibex Converter       =        0.018036 sec
                 beta     gamma    S0    I0   R0    N         runtime
point-based  0.201544  0.071086  0.99  0.01  0.0  1.0  0:04:47.749004


In [27]:
from datetime import timedelta, datetime

total= "0:04:47.749004"
sim = "0:00:02.754625"
format = "%H:%M:%S.%f"

nonsim = datetime.strptime(total, format) - datetime.strptime(sim, format)
print(f"{MAX_TIME}\t{sim}\t{nonsim}")

250	0:00:02.754625	0:04:44.994379


In [ ]:
# Solve using interval-based method

funman_request = get_request()
setup_common(funman_request, NUM_STEPS, STEP_SIZE, point_based=False, synthesize=False)
# add_unit_test(funman_request)
results = run(funman_request)
report(results, "interval-based")

[1000.00000, 1000.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
2024-08-23 15:08:26,279 - funman.server.worker - INFO - FunmanWorker running...
2024-08-23 15:08:26,282 - funman.server.worker - INFO - Starting work on: 100f7f32-6527-426f-8d4e-62a447cad847
2024-08-23 15:08:

               beta gamma      S0   I0   R0         runtime
point-based     0.0  0.14  1000.0  1.0  0.0  0:00:01.710106
interval-based  0.0  0.14  1000.0  1.0  0.0  0:00:01.612061


In [ ]:
results.timing

FunmanResultsTiming(start_time=datetime.datetime(2024, 8, 23, 15, 8, 26, 282907), end_time=datetime.datetime(2024, 8, 23, 15, 8, 27, 894968), total_time=datetime.timedelta(seconds=1, microseconds=612061), solver_time=None, encoding_time=None, progress_timeseries=[(datetime.datetime(2024, 8, 23, 15, 8, 27, 886926), 0.0)])